In [1]:
#import block
import numpy as np
from scipy.special import hankel1 as besselh
import scipy.linalg as linalg
from numpy import log, pi, sqrt

In [2]:
#variable block
h = 1.001
d = [0.5, 0.25]
a = [0.5, 1]
heaving = [0, 1]
m0s = [0.0001, 0.001, 0.01]
g = 9.81
rho = 1023

In [3]:
#useful constants block
bds = len(a) # number of boundaries

In [4]:
def particular_potential(r, n):
    if n >= bds: # free surface region
        return 0
    if heaving[n]:
        return - (r**2)/(4 * (h - d[n]))
    else:
        return 0

def diff_particular_potential_total(r, n):
    if n >= bds: # free surface region
        return 0
    if heaving[n]:
        return - r/2
    else:
        return 0

def b_potential_entry(n):
    return - particular_potential(a[n], n) + particular_potential(a[n], n + 1)

def b_velocity_entry(n):
    return - diff_particular_potential_total(a[n], n) + diff_particular_potential_total(a[n], n + 1)
    
# sets potential equation for nth boundary into the A matrix
def potential_row(A, n, m0):
    if n == 0:
        if bds == 1: # single cylinder
            A[0][0] = 1
            A[0][1] = besselh(0, m0 * a[0])
        else:
            A[0][0] = 1
            A[0][1] = 1
            A[0][2] = log(a[0])
    elif n == bds - 1:
        A[n][2*n - 1] = 1
        A[n][2*n] = log(a[n])
        A[n][2*n + 1] = besselh(0, m0 * a[n])
    else:
        A[n][2*n - 1] = 1
        A[n][2*n] = log(a[n])
        A[n][2*n + 1] = 1
        A[n][2*n + 2] = log(a[n])

def velocity_row(A, n, m0):
    if n == 0:
        if bds == 1: # single cylinder
            A[bds][0] = 0
            A[bds][1] = - h * m0 * besselh(1, m0 * a[0])
        else:
            A[bds][0] = 0
            A[bds][1] = 0
            A[bds][2] = 1/a[0] * (h - d[1])
    elif n == bds - 1:
        A[bds + n][2*n - 1] = 0
        A[bds + n][2*n] = 1/a[n] * (h - d[n])
        A[bds + n][2*n + 1] = - h * m0 * besselh(1, m0 * a[n])
    else:
        A[bds + n][2*n - 1] = 0
        A[bds + n][2*n] = 1/a[n] * (h - d[n])
        A[bds + n][2*n + 1] = 0
        A[bds + n][2*n + 2] = 1/a[n] * (h - d[n + 1])

def build_A(m0):
    A = np.zeros((2 * bds, 2 * bds), dtype = complex)
    for n in range(bds):
        potential_row(A, n, m0)
        velocity_row(A, n, m0)
    return A

def build_B():
    b1 = (np.vectorize(b_potential_entry, otypes = [float]))(list(range(bds)))
    b2 = (np.vectorize(b_velocity_entry, otypes = [float]))(list(range(bds)))
    return np.concatenate([b1, b2])
    

# modifies A matrix for a particular m0, all other parameters the same.
def A_new_m0(A, m0):
    A[bds-1][2*bds-1] = besselh(0, m0 * a[-1]) 
    A[2*bds-1][2*bds-1] = - h * m0 * besselh(1, m0 * a[-1])
    

In [13]:
# hydro coefficient calculation functions
def particular_potential_int_eval(region, boundary):
    return (a[boundary]**2)/(16 * (h - d[region]))

def total_particular():
    accumulator = 0
    for region in range(bds):
        if heaving[region]:
            if region == 0:
                accumulator += particular_potential_int_eval(0, 0)
            else:
                accumulator += particular_potential_int_eval(region, region) - particular_potential_int_eval(region, region - 1)
    return accumulator

#dphi/dz mandated to be 1 in the heaving regions, so just integrate potential * r

def ln_potential_int_eval(bd):
    return (a[bd]**2/2) * (log(a[bd]) - 1/2)

def const_potential_int_eval(bd):
    return (a[bd]**2/2)

def create_c_vector():
    c = []
    for region in range(bds):
        if region == 0:
            if heaving[0]:
                c.append(const_potential_int_eval(0))
            else:
                c.append(0)
        else:
            if heaving[region]:
                c.append(const_potential_int_eval(region) - const_potential_int_eval(region - 1))
                c.append(ln_potential_int_eval(region) - ln_potential_int_eval(region - 1))
            else:
                c.append(0)
                c.append(0)
    return c

def get_hydro_coeffs(X, m0):
    const = total_particular()
    c = create_c_vector()
    raw = np.dot(X, c) + const
    total = 2 * pi * rho * raw * h**3 
    return np.real(total), (np.imag(total)) # * omega_from_m0(m0))

def to_nondim(coeff, a_norm):
    return coeff/(pi * a_norm**3 * rho)

def omega_from_m0(m0): # at small m0 approximation
    return sqrt(m0**2 * h * g)

def get_max_heaving_radius():
    max_rad = a[0]
    for i in range(bds - 1, 0, -1):
        if heaving[i]:
            max_rad = a[i]
            break
    return max_rad

In [6]:
A = build_A(m0s[0])
b = build_B()

solutions = []

for m0 in m0s:
    A_new_m0(A, m0)
    solutions.append(linalg.solve(A,b))

In [14]:
regular_hydros = []
nondim_hydros = []
a_norm = get_max_heaving_radius()
for i in range(len(m0s)):
    added_mass, damping = get_hydro_coeffs(solutions[i][:-1], m0s[i])
    regular_hydros.append((added_mass, damping))
    nondim_hydros.append((to_nondim(added_mass, a_norm), to_nondim(damping, a_norm)))
print(regular_hydros)
print(nondim_hydros)

[(15393.495061672578, 2371.126784863897), (11917.687922270203, 2371.109178341888), (8440.072501356692, 2369.8895160061966)]
[(4.789737694087725, 0.737784063555586), (3.7082286276752345, 0.7377785852271239), (2.6261569083966667, 0.737399083194615)]


In [15]:
print((- a[0]/(2*h) * log (m0s * a[1])))
print(pi * a[0] / (4 * h))
print(-a[1]/(2*h)*(1 - (a[0]/a[1])**2)**2*log(m0s * a[1]))
print(pi * a[1] / (4 * h) * (1 - (a[0]/a[1])**2)**2)

[2.30028481 1.72521361 1.1501424 ]
0.3923067749238004
[2.58782041 1.94086531 1.2939102 ]
0.44134512178927543


In [9]:
print(a_norm)

1
